<a href="https://colab.research.google.com/github/brenoslivio/SME0828_DataScience/blob/master/Projects/3%20-%20Classifica%C3%A7%C3%A3o/Projeto3_Classificacao_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SME0828 - Introdução à Ciência de Dados
# Projeto 3: Classificação 

### Alunos

Aline Fernanda da Conceição, Nº USP: 9437275

Breno Lívio Silva de Almeida, Nº USP: 10276675

Matheus Victal Cerqueira, Nº USP: 10276661

***Universidade de São Paulo, São Carlos, Brasil.***

## Exercícios sugeridos para a atividade avaliativa 3

Os seguintes exercícios foram sugeridos para entrega em 03 de novembro de 2020 para a disciplina SME0828 - Ciência de Dados oferecida pelo Instituto de Ciências Matemáticas e de Computação (ICMC) da Universidade de São Paulo (USP), a qual foi ministrada pelo professor Francisco Aparecido Rodrigues. Vamos trabalhar com o conjunto de dados [*Star dataset to predict star types*](https://www.kaggle.com/deepu1109/star-dataset). O dataset tem várias informações sobre características de estrelas como temperatura absoluta, luminosidade relativa, tipos de estrelas, entre outros. Vamos classificar os dados considerando vários classificadores conhecidos.



---



## Sumário

1. Desenvolvimento

  1.1 [Exercício 1](#ex1)

  1.2 [Exercício 2](#ex2)

  1.3 [Exercício 3](#ex3)

  1.4 [Exercício 4](#ex4)

  1.5 [Exercício 5](#ex5)

  1.6 [Exercício 6](#ex6)



---



## Desenvolvimento



### <a name="ex1"></a> 1 - Considere a base de dados sobre doenças cardíacas:https://www.kaggle.com/ronitf/heart-disease-uci 
### Faça o pré-processamento dos dados e classifique os pacientes de acordo com a variável “target”.  Considere os classificadores: Bayesiano paramétrico, Bayesiano não-paramétrico e Naive Bayes.

#### Pré-processamento dos dados

Foi considerada a base de dados de estrelas [*Star dataset to predict star types*](https://www.kaggle.com/deepu1109/star-dataset) para o exercício. Vamos carregar seus dados considerando informações que serão acrescentadas como os tipos de estrelas que serão explicado no código.

In [7]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import statistics

dataStar = pd.read_csv('https://raw.githubusercontent.com/brenoslivio/SME0828_DataScience/master/Projects/data/starsDataset.csv', 
        dtype={
          "Temperature (K)": np.int64,
          "Luminosity(L/Lo)": np.float64,
          "Radius(R/Ro)": np.float64,
          "Absolute magnitude(Mv)": np.float64,
          "Star type": str,
          "Star color": str,
          "Spectral Class": str
        },
        na_values="",)

dataStar.replace("", np.nan, inplace=True)

dataStar.dropna(inplace=True)

# Já vamos colocar os labels adequados para os tipos de estrelas que não tem de padrão no dataset

# star class:  Star Type
#     0     :  Brown Dwarf
#     1     :  Red Dwarf
#     2     :  White Dwarf
#     3     :  Main-Sequence
#     4     :  Supergiant
#     5     :  Hypergiant

dataStar["Star type"] = dataStar["Star type"].str.replace("0", "Brown Dwarf")
dataStar["Star type"] = dataStar["Star type"].str.replace("1", "Red Dwarf")
dataStar["Star type"] = dataStar["Star type"].str.replace("2", "White Dwarf")
dataStar["Star type"] = dataStar["Star type"].str.replace("3", "Main-Sequence")
dataStar["Star type"] = dataStar["Star type"].str.replace("4", "Supergiant")
dataStar["Star type"] = dataStar["Star type"].str.replace("5", "Hypergiant")

dataStar.sample(5)

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
82,8930,0.00056,0.0095,13.78,White Dwarf,white,A
20,25000,0.05600,0.0084,10.58,White Dwarf,Blue White,B
197,3496,0.00125,0.3360,14.94,Red Dwarf,Red,M
54,3650,310000.00000,1324.0000,-7.79,Hypergiant,Red,M
71,3607,0.02200,0.3800,10.12,Red Dwarf,Red,M


Temos três variáveis `Star type`, `Star color`, `Spectral Class` que podemos possivelmente indicar como labels para fazer a classificação, mas precisamos ter uma noção da frequência absoluta dos valores de cada variável.

In [8]:
dataStar['Star type'].value_counts()

Red Dwarf        40
Brown Dwarf      40
Supergiant       40
Main-Sequence    40
Hypergiant       40
White Dwarf      40
Name: Star type, dtype: int64

In [9]:
dataStar['Star color'].value_counts()

Red                   112
Blue                   55
Blue-white             26
Blue White             10
yellow-white            8
White                   7
Yellowish White         3
Blue white              3
white                   3
Orange                  2
yellowish               2
Whitish                 2
Blue-White              1
Orange-Red              1
Blue white              1
Blue                    1
White-Yellow            1
Yellowish               1
Pale yellow orange      1
Name: Star color, dtype: int64

In [10]:
dataStar['Spectral Class'].value_counts()

M    111
B     46
O     40
A     19
F     17
K      6
G      1
Name: Spectral Class, dtype: int64

Definitivamente o tipo de estrela seria a melhor opção de classificação considerando que alguns valores nas outras variáveis aparecem somente uma vez, o que não teria sentido para conseguir validar um possível aprendizado. Vamos pegar as classes nessa variável `Star type`.

In [11]:
# Nomes das classes
classes = np.array(pd.unique(dataStar[dataStar.columns[-3]]), dtype=str)  

# Atributos do dataset
attributes = list(dataStar.columns)

classes

array(['Brown Dwarf', 'Red Dwarf', 'White Dwarf', 'Main-Sequence',
       'Supergiant', 'Hypergiant'], dtype='<U13')

Dependendo da relação dessas variáveis que foram descartadas como possível label, podemos até usar usando a função do Pandas [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) para transformar facilmente em valores binários por coluna. Entretanto, para o caso desse dataset, elas impactam negativamente na classificação, por isso iremos removê-las.

In [12]:
dataConcat = dataStar.copy()

# Os comentários servem apenas para se quiser verificar como essas outras variáveis acabam
# tendo uma influência até negativa na classificação

#Spectral = pd.get_dummies(dataConcat['Spectral Class'], drop_first=True)
dataConcat.drop('Spectral Class',axis=1,inplace=True)
#dataConcat = pd.concat([dataConcat, Spectral],axis=1)

#Color = pd.get_dummies(dataConcat['Star color'], drop_first=True)
dataConcat.drop('Star color',axis=1,inplace=True)
#dataConcat = pd.concat([dataConcat, Color],axis=1)

dataConcat.sample(5)

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type
127,2861,0.000190,0.0899,16.71,Brown Dwarf
29,7230,0.000080,0.0130,14.08,White Dwarf
186,2968,0.000461,0.1190,17.45,Brown Dwarf
187,3484,0.000551,0.0998,16.67,Brown Dwarf
107,12893,184000.000000,36.0000,-6.34,Supergiant


Vamos converter o data frame para um array numpy com a intenção de manipular melhor com bibliotecas próprias para a classificação.

In [13]:
data = dataConcat.to_numpy()
data

array([[3068, 0.0024, 0.17, 16.12, 'Brown Dwarf'],
       [3042, 0.0005, 0.1542, 16.6, 'Brown Dwarf'],
       [2600, 0.0003, 0.102, 18.7, 'Brown Dwarf'],
       ...,
       [8829, 537493.0, 1423.0, -10.73, 'Hypergiant'],
       [9235, 404940.0, 1112.0, -11.23, 'Hypergiant'],
       [37882, 294903.0, 1783.0, -7.8, 'Hypergiant']], dtype=object)

In [14]:
dataTrain = data
y = dataTrain[:, 4] # Pega o label que iremos classificar, o tipo de estrela
X = dataTrain[:, 0:4] # Pega os 4 valores relevantes que sobraram para a classificação das estrelas

Vamos normalizar os dados para evitar o efeito da escala dos atributos envolvidos.

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

Por fim, separando os dados de forma que tenha o conjunto de dados para treino e o conjunto de dados para teste para poder testar se o treino foi efetivo. Ou seja, esses dados servem para ver se foi possível generalizar o suficiente considerando que eles não estavam no conjunto de dados para treino, sendo totalmente novos.

In [16]:
from sklearn.model_selection import train_test_split
p = 0.8 # Fração de elementos que iremos usar no treinamento
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 42)

#### Classificador Bayesiano

Vamos realizar propriamente a classificação que utiliza propriamente da ideia de encontrar a função densidade de probabilidade da [distribuição normal multivariada](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.multivariate_normal.html) do conjunto X, os dados escolhidos para o treino.

In [17]:
from scipy.stats import multivariate_normal

# P irá guardar a probabilidade para as linhas e colunas
P = pd.DataFrame(data = np.zeros((x_test.shape[0], len(classes))), columns = classes) 

Pc = np.zeros(len(classes)) # A fração de elementos em cada classe
for i in np.arange(0, len(classes)):
    elements = tuple(np.where(y_train == classes[i]))
    Pc[i] = len(elements)/len(y_train)
    Z = x_train[elements,:][0]
    m = np.mean(Z, axis = 0)
    cv = np.cov(np.transpose(Z)) # Matriz de covariância da distribuição
    for j in np.arange(0,x_test.shape[0]):
        x = x_test[j,:]
        pj = multivariate_normal.pdf(x, mean=m, cov=cv, allow_singular=True)
        P[classes[i]][j] = pj*Pc[i]

Gerado o modelo podemos testar com a distribuição guardada para testes com o intuito de validar o modelo e verificar sua acurácia.

In [18]:
from sklearn.metrics import accuracy_score

y_pred = []

for i in np.arange(0, x_test.shape[0]):
    c = np.argmax(np.array(P.iloc[[i]])) # Pega a classe/coluna com maior chance de ser
    y_pred.append(classes[c])

y_pred = np.array(y_pred, dtype=str)

score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

Accuracy: 0.9791666666666666


#### Classificador Bayesiano não-paramétrico

Com um estimador não-paramétrico, consideramos que não conhecemos a função de distribuição dos nossos dados, não conhecendo os parâmetros envolvidos. Praticamente vai envolver a criação e convolução de distribuições normais acerca de um conjunto dos nossos dados com a intenção de achar curvas que irão ajudar na classificação. Vamos usar do kernel de gaussiana da [biblioteca do sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html).

In [25]:
from sklearn.neighbors import KernelDensity

P = pd.DataFrame(data=np.zeros((x_test.shape[0], len(classes))), columns = classes) 
Pc = np.zeros(len(classes)) 
h = 1 # Esse valor será melhor discutido no próximo exercício
for i in np.arange(0, len(classes)): 
    elements = tuple(np.where(y_train == classes[i])) 
    Pc[i] = len(elements)/len(y_train) 
    Z = x_train[elements,:][0] 
    kde = KernelDensity(kernel='gaussian', bandwidth=h).fit(Z)
    for j in np.arange(0,x_test.shape[0]): 
        x = x_test[j,:]
        x = x.reshape((1,len(x)))
        pj = np.exp(kde.score_samples(x)) 
        P[classes[i]][j] = pj*Pc[i]

A variável `h` usada no código tem uma relevância importante com a possibilidade de formarmos curvas suscetíveis a ruídos, menos suave (overfitting), sendo valores mais baixos, e também curvas que podemos perder mais dados, mais suave (underfitting), sendo valores mais altos. Por isso devemos sempre focar em achar um valor razoável. Isso será melhor discutido no próximo exercício usando desse mesmo conjunto de dados.

Vamos, por fim, calcular a acurácia da forma que fizemos anteriormente.

In [26]:
from sklearn.metrics import accuracy_score

y_pred = [] 

for i in np.arange(0, x_test.shape[0]):
    c = np.argmax(np.array(P.iloc[[i]]))
    y_pred.append(classes[c])

y_pred = np.array(y_pred, dtype=str)

score = accuracy_score(y_pred, y_test)

print('Accuracy:', score)

Accuracy: 0.8958333333333334


### <a name="ex2"></a> 2 - No classificar não-paramétrico, verifique o efeito do hiperparâmetro h na classificação dos dados de diabetes, encontrando seu melhor valor: https://www.kaggle.com/uciml/pima-indians-diabetes-database
